In [46]:
import requests
import base64  #Required for client creds
import datetime as dt
from urllib.parse import urlencode

In [2]:
client_id =  '3f4aa9f9ddcf4691b2f8e1ca719d3ebe'
client_secret = 'd063c88911434a9c8bf8bc95f1e7b537'

In [3]:
client_creds = base64.b64encode(f"{client_id}:{client_secret}".encode())
print(client_creds)

b'M2Y0YWE5ZjlkZGNmNDY5MWIyZjhlMWNhNzE5ZDNlYmU6ZDA2M2M4ODkxMTQzNGE5YzhiZjhiYzk1ZjFlN2I1Mzc='


In [4]:
#get token lookup
token_url = 'https://accounts.spotify.com/api/token'
method = 'POST'
token_headers = {
    'Authorization':f'Basic {client_creds.decode()}'
}

In [5]:
r=requests.post(token_url,data=token_data,headers=token_headers)
valid_request = r.status_code in range(200,299)


{'access_token': 'BQBGb0-G4nGNkrArJTwKDGjzSyuxSIsEfwWRZl5lwEgasy-LBi6Adc2qA1wbAz4GnHqMKjxTiYwW1hPSklk', 'token_type': 'Bearer', 'expires_in': 3600, 'scope': ''}


In [6]:
if valid_request:
    response = r.json()
    print(response)
    access_token = response['access_token']
    expires = dt.datetime.now()+dt.timedelta(seconds=response['expires_in'])

In [54]:
class SpotifyAPI(object):
    access_token,client_id,client_secret = None,None,None
    expires_at = dt.datetime.now()
    expired = True
    def __init__(self,client_id,client_secret,*args,**kwargs):
        super().__init__(*args,**kwargs)
        self.client_id,self.client_secret = client_id,client_secret
   
    def get_client_creds(self):
        client_id = self.client_id
        client_secret = self.client_secret
        if client_id is None or client_secret is None:
            raise Exception('You must set client id and secret')
        client_creds = base64.b64encode(f"{client_id}:{client_secret}".encode())
        return client_creds.decode()
    def get_token_headers(self):
        client_creds_b64 = self.get_client_creds()
        return{
            'Authorization':f"Basic {client_creds_b64}"
        }
    def get_token_data(self):
        return {
            'grant_type':'client_credentials'
        }
    def get_auth_token(self):
        if access_token== None or self.expired:
            self.authorize()
        return self.access_token
        
    def authorize(self):
        r=requests.post(token_url,data=self.get_token_data(),headers=self.get_token_headers())
        if r.status_code in range(200,299):
            response = r.json()
            access_token = response['access_token']
            expires = dt.datetime.now()+dt.timedelta(seconds=response['expires_in'])
            self.expires_at = expires
            self.expired = dt.datetime.now()>expires
            self.access_token = access_token
            return True 
        return False
    def search(self,params):
        headers = {"Authorization": "Bearer {}".format(self.get_auth_token())}
        endpoint = 'https://api.spotify.com/v1/search'
        lookup_url = endpoint+'?'+'&'.join(['{}={}'.format(param,val) for param,val in params.items()])
        r = requests.get(lookup_url,headers=headers)
        if r.status_code in range(200,299):
            return r.json()
        else:
            return None

In [57]:
client = SpotifyAPI(client_id,client_secret)

In [58]:
au = client.get_auth_token()

In [59]:
vasd = requests.get('https://api.spotify.com/v1/me',headers ={"Authorization": "Bearer {}".format(au)} )

In [61]:
vasd.text

'{\n  "error" : {\n    "status" : 401,\n    "message" : "Unauthorized."\n  }\n}'

In [ ]:
spotify:user:1228617180

In [56]:
client.search({'q':'Lana Del Ray','type':'artist'})

{'artists': {'href': 'https://api.spotify.com/v1/search?query=Lana+Del+Ray&type=artist&offset=0&limit=20',
  'items': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/00FQb4jTyendYWaN8pK0wa'},
    'followers': {'href': None, 'total': 10804104},
    'genres': ['art pop', 'pop'],
    'href': 'https://api.spotify.com/v1/artists/00FQb4jTyendYWaN8pK0wa',
    'id': '00FQb4jTyendYWaN8pK0wa',
    'images': [{'height': 640,
      'url': 'https://i.scdn.co/image/6e8dc460cfb6b89d7970302259febd0aa73b38c6',
      'width': 640},
     {'height': 320,
      'url': 'https://i.scdn.co/image/bcdee6b17410b5d27d40bc114757e38125b00c84',
      'width': 320},
     {'height': 160,
      'url': 'https://i.scdn.co/image/0c50e5ae217b7a86f9f5114dc5e0f1beb8e9185d',
      'width': 160}],
    'name': 'Lana Del Rey',
    'popularity': 85,
    'type': 'artist',
    'uri': 'spotify:artist:00FQb4jTyendYWaN8pK0wa'},
   {'external_urls': {'spotify': 'https://open.spotify.com/artist/1IoH5ykVwG4K5c98iAhQYk'},


In [32]:
client.authorize()

<Response [200]>
{'access_token': 'BQAcSjL4Zsp0VgXpKlcQR9DQCcOiXHh0FkZrIVhITlAeszzDYZsMVk3_grzjRDqvzDNMXgStwcs6dCQq8zE', 'token_type': 'Bearer', 'expires_in': 3600, 'scope': ''}


True

In [33]:
client.access_token

'BQAcSjL4Zsp0VgXpKlcQR9DQCcOiXHh0FkZrIVhITlAeszzDYZsMVk3_grzjRDqvzDNMXgStwcs6dCQq8zE'

In [48]:
headers = {
    "Authorization": f"Bearer {access_token}"
}
endpoint = 'https://api.spotify.com/v1/search'
params = {"q":"Deportivo","type":"track"}
lookup_url = endpoint+'?'+'&'.join(['{}={}'.format(k,v) for k,v in params.items()])

r = requests.get(lookup_url,headers=headers)

In [49]:
r.json()

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=Deportivo&type=track&offset=0&limit=20',
  'items': [{'album': {'album_type': 'single',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/5J7rXWjtn5HzUkJ4Jet8Fr'},
       'href': 'https://api.spotify.com/v1/artists/5J7rXWjtn5HzUkJ4Jet8Fr',
       'id': '5J7rXWjtn5HzUkJ4Jet8Fr',
       'name': 'Alvaro Diaz',
       'type': 'artist',
       'uri': 'spotify:artist:5J7rXWjtn5HzUkJ4Jet8Fr'},
      {'external_urls': {'spotify': 'https://open.spotify.com/artist/6w3SkAHYPsQ1bxV7VDlG5y'},
       'href': 'https://api.spotify.com/v1/artists/6w3SkAHYPsQ1bxV7VDlG5y',
       'id': '6w3SkAHYPsQ1bxV7VDlG5y',
       'name': 'Cazzu',
       'type': 'artist',
       'uri': 'spotify:artist:6w3SkAHYPsQ1bxV7VDlG5y'},
      {'external_urls': {'spotify': 'https://open.spotify.com/artist/11usEAQkFE6UJwsyAKkchi'},
       'href': 'https://api.spotify.com/v1/artists/11usEAQkFE6UJwsyAKkchi',
       'id': '11usEAQkFE6UJwsyAKk